In [1]:
import requests
import random as rd
import re
import time
import pandas as pd
from datetime import datetime
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm

# 상의
수집 데이터
카테고리, 아이템품번, 아이템 이름, 한정판매, 단독판매, 색상, 타겟 성별, 구매 성별, 구매 나이, 1개월 조회수, 좋아요 수, 가격(회원가), 배송정보, 평점, 후기수, Q&A 수, 1년 누적 판매량

In [ ]:
# driver.close()

In [3]:
# 상의 페이지(1 page)
driver = webdriver.Chrome()
driver.get("https://www.musinsa.com/categories/item/002?d_cat_cd=001&brand=&list_kind=small&sort=pop_category&sub_sort=&page={}&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=".format(1))
driver.maximize_window()

In [4]:
# 제품 선택 (임의로 1의 제품을 선택하여 수집)
first_content = driver.find_element(By.CSS_SELECTOR, '#searchList > li:nth-child(1) > div.li_inner > div.article_info > p.list_info > a')
first_content.click()
# searchList > li:nth-child(9) > div.li_inner > div.article_info > p.item_title > a
# li:nth-child(9) 여기서 괄호 안에 숫자가 제품의 추천순

In [10]:
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

#### 대분류, 중분류

In [11]:
category1 = soup.select_one('div.product-detail__sc-up77yl-0.htaeEr').get_text().split('\xa0')[0]
category1

'아우터'

In [12]:
category2 = soup.select_one('div.product-detail__sc-up77yl-0.htaeEr').get_text().split('\xa0')[2]
category2

'숏패딩/숏헤비 아우터'

#### 아이템 이름

In [13]:
# <span class="product_title"><em>하트 벌룬 스웻 셔츠-레드</em><span class="product_title_eng">Heart Balloon Sweat Shirt-Red</span> </span>
name = soup.select_one('h3.product-detail__sc-1klhlce-3.fjguJZ').get_text()
name

'베이직 푸퍼_SPJPE11C11'

#### 아이템 품번

In [14]:
def 품번():
    overlap = []
    for i in soup.select('.product-detail__sc-achptn-4.flVcwF'):
        overlap.append(i.get_text())

    if '품번' in overlap:
        index_num = overlap.index('품번')

        if overlap[index_num+1][0] == ' ':
            return overlap[index_num+1][1:]
        return overlap[index_num+1]
    else: return None

품번()

'SPJPE11C11'

### 한정판매, 무신사 단독 통합
0: False, 1: True

In [15]:
# 해당 태그가 없는 상품이 있을까봐 예외처리
def 한정_단독():
    '''
    return: limited_sales, exclusive_sale\n(한정판매 여부, 단독 여부) 튜플입니다.
    '''
    try:
        sales = soup.select_one('h4.product-detail__sc-1qpt7wc-1.dbnyPg').get_text()
        if sales == '한정판매':
            limited_sales = 1
        else:
            limited_sales = 0

        if sales == '무신사 단독':
            exclusive_sale = 1
        else:
            exclusive_sale = 0

    except:
        limited_sales = 0
        exclusive_sale = 0

    return limited_sales, exclusive_sale

한정_단독()

(1, 0)

### 타겟 성별
- 남자 = 0
- 여자 = 1
- 남/여 = 2

In [16]:
def 성별():
    overlap = []
    for i in soup.select('.product-detail__sc-achptn-4.flVcwF'):
        overlap.append(i.get_text())
    if '성별' in overlap:
        index_num = overlap.index('성별')
        # return '아아',overlap[index_num])
        if overlap[index_num-1] == '시즌':
            return overlap[index_num+2]
        else: return overlap[index_num+1]
    else: return None

def 성별2():
    if 성별() is None:
        return 성별()
    else:
        if 성별() == '남성':
            return  0
        elif 성별() == '여성':
            return  1
        else: return  2
성별2()

0

### 구매성별
- 남 = 0
- 여 = 1

In [17]:
# man graph_set_text
# 남성일 때와 여성일 때 태그가 다름
def 구매성별():
    try:
        if soup.select_one('em.product-detail__sc-wep3gk-3.kJJbYq').get_text() == '남성':
            buy_gender = 0
    except:
        try:
            if soup.select_one('em.product-detail__sc-wep3gk-3.fdjMAR').get_text() == '여성':
                buy_gender = 1
        except:   #항목 자체가 없을 때
            buy_gender = None

    return buy_gender

구매성별()

0

### 구매 나이

In [18]:
def 구매나이():
    '''
    return : (age_name_1, age_name_2) 튜플입니다.
    '''

    try:
        # 해당 엘리먼트가 있는 곳까지 스크롤
        target_element = driver.find_element(By.XPATH, '//*[@id="page_product_detail"]/div[2]/div[7]/ul/li[1]/h3')   # 구매나이 클릭
        driver.execute_script("arguments[0].scrollIntoView(true);", target_element)
        target_element.click()

        respone = driver.page_source
        soup = BeautifulSoup(respone, 'html.parser')

        age_name = soup.select('ul.bar_wrap dl:has(span.bar.hightest) dt.bar_name')

        try:
            if age_name[0] in age_name or age_name[1] in age_name:
                age_name_1 = ','.join(age_name[0]).split()
                age_name_1 = age_name_1[0]
                age_name_2 = ','.join(age_name[1]).split()
                age_name_2 = age_name_2[0]

        except:
            age_name_2 = 0
            # age_name[0]은 필히 존재 하고 age_name[1]의 유무로 인하여 오류가 발생하므로
            # 오류 발생 시 age_name[1]의 텍스트 값인 age_name_2만 '0' 으로 설정
    except:
        age_name_1 = 0
        age_name_2 = 0

    return age_name_1, age_name_2

구매나이()

('19~23세', 0)

### 1개월 조회수

In [19]:
def 조회수():
    overlap = []
    for i in soup.select('.product-detail__sc-achptn-4.flVcwF'):
        overlap.append(i.get_text())

    if '조회수(1개월)' in overlap:
        index_num = overlap.index('조회수(1개월)')
        sale = overlap[index_num+1][:-4]

        if sale[-2] =='만':
            return float(sale[:-2]) * 10000
        elif sale[-2] =='천':
            return float(sale[:-2]) * 1000
        else: return float(sale)
    else: return None

def 조회수2():
    some_tag = driver.find_element(By.CSS_SELECTOR, '#page_product_detail > div.right_area.page_detail_product > div.section_graph_detail > ul > li.graph_title_viewcount > h3')
    action = ActionChains(driver)
    action.move_to_element(some_tag).perform()

    first_content = driver.find_element(By.CSS_SELECTOR, '#page_product_detail > div.right_area.page_detail_product > div.section_graph_detail > ul > li.graph_title_viewcount > h3')
    first_content.click()

    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')

    view = soup.select_one('ul.graph_sub_title>li>h4>strong').get_text().split(' 회')[0]

    if view[-1] =='만':
        return float(view[:-1]) * 10000
    elif view[-1] =='천':
        return float(view[:-1]) * 1000
    else: return float(view.split('회')[0])

try: print(조회수2())
except: print(조회수())

547000.0


### 좋아요 수

In [20]:
def 좋아요():
    try:
        tmp_like = int(soup.select_one('span.product-detail__sc-achptn-4.flUHrZ').get_text().replace(',', ''))
    except:
        tmp_like = 0

    return tmp_like

좋아요()

127438

### 가격

In [21]:
def 가격():
    #일반
    try:
        price_element = soup.select('div.product-detail__sc-1p1ulhg-6.bIRSux span.product-detail__sc-1p1ulhg-7.hSwsZE')
        price_element = ','.join(price_element[0]).split()

        if len(price_element) == 3: # 30,000원 ~ 40,000원
            price = float(price_element[2].replace(',', '').replace('원',''))
        else: # 40,000원
            price = float(price_element[0].replace(',', '').replace('원',''))
    #클리어런스
    except:
        price = float(soup.select_one('span.product-detail__sc-1p1ulhg-7.kqpaoP').get_text()[:-1].replace(',', ''))

    return price
가격()

62910.0

### 할인율

In [22]:
def 할인율():
    if soup.select_one('div.product-detail__sc-1p1ulhg-14.gNkVLj')!=None:
        discount=soup.select_one('div.product-detail__sc-1p1ulhg-14.gNkVLj').get_text().split('%')[0]

    else :
        discount = 0

    return discount
할인율()

'10'

### 배송 일수

In [23]:
def get_date_difference(estimated_date):
    date_format = '%Y/%m/%d'  # 대문자 Y는 연도를 4자리로 표시
    target_date = datetime.strptime(estimated_date, date_format)

    # 오늘 날짜를 구합니다.
    today = datetime.now()

    # 날짜 차이를 계산합니다.
    difference = today - target_date

    return abs(difference.days)

def 배송일자():
        arrival_text = soup.select_one(".product-detail__sc-1ys1x0v-4.bPYIon")
        if arrival_text is not None:
            date_text = arrival_text.get_text().split('(')[0]
            estimated_date = '2024/' + date_text

            return  get_date_difference(estimated_date)

        else:
            # 배송 예정일이 없을 경우
            a = []
            for i in soup.select('span.product-detail__sc-1ts9zk8-4.hwqlaH'):
                a.append(i.get_text())

            try:
                if '출고 정보' in a:
                    print('출고 정보 + 2 :')
                    num = a.index('출고 정보') # 0번 # '출고 정보'가 없을 경우 에러가 뜸!
                    return  int(a[num+1].split(' ')[1].split('일')[0]) + 2
                else:
                    return  None
            except:
                 print('출고 정보에 날짜 없음(예: 13시 까지 주문 시 당일 출고)')
                 return None

배송일자()

6

### 평점

In [24]:
def 평점():
    try:
        tmp_pro_rating = float(soup.select_one('.product-detail__sc-achptn-4.iKKecn').get_text())
    except:
        tmp_pro_rating = None

    return tmp_pro_rating

평점()

4.9

### 후기 수

In [25]:
def 후기수():
    try:
        tmp_review = int(soup.select_one('#review_total').get_text().replace(',', ''))
    except:
        tmp_review = None

    return tmp_review

후기수()

42739

### 구매수

In [26]:
def 누적판매():
    overlap = []
    for i in soup.select('.product-detail__sc-achptn-4.flVcwF'):
        overlap.append(i.get_text())

    if '누적판매(1년)' in overlap:
        index_num = overlap.index('누적판매(1년)')
        sale = overlap[index_num+1][:-4]
        if sale[-2] =='만':
            return float(sale[:-2]) * 10000
        elif sale[-2] =='천':
            return float(sale[:-2]) * 1000
        else: return float(sale)
    else: return None

누적판매()

76000.0

### 리뷰 크롤링1

In [27]:
# 리뷰 크롤링 함수1
def review_crawling(soup):

    # 유저명
    tmp_user_names = []

    for i in soup.select('p.review-profile__name'):
        if i.get_text() == '탈퇴회원':
            tmp_result = '탈퇴회원'

        else:
            tmp_result = i.get_text().split(' ', maxsplit=1)[1]
        tmp_user_names.append(tmp_result)

    # 유저 평점
    tmp_user_ratings = []
    search_review_rating = re.compile('\d+')

    for i in soup.select('span.review-list__rating__active'):
        tmp_result = search_review_rating.findall(str(i))[0]
        tmp_user_ratings.append(tmp_result)

    # 유저 리뷰
    tmp_user_reviews = []

    for i in soup.select('div.review-contents__text'):
        tmp_result = i.get_text()
        tmp_user_reviews.append(tmp_result)

    # 상품이름
    name = soup.select_one('h3.product-detail__sc-1klhlce-3.fjguJZ').get_text()
    tmp_names = [name] * len(tmp_user_ratings)

    # 품번
    number = soup.select_one('li.product-detail__sc-achptn-2.dEZkTF > div.product-detail__sc-achptn-6.gfoaTb > span.product-detail__sc-achptn-4.flVcwF').get_text()
    if number[0] == ' ':   # 앞 자리 공백 제거
        number = number[1:]
    tmp_numbers = [number] * len(tmp_user_ratings)

    return tmp_names, tmp_numbers, tmp_user_names, tmp_user_ratings, tmp_user_reviews

### 리뷰 크롤링2

In [29]:
# 리뷰 크롤링 함수2 (함수1 사용)
def get_tmp_df_musinsa_review_raw(driver, tag_dict):
    '''
    ex) tag_dict = {'name1':'tag1', ...}

    '''
    tmp_names = []
    tmp_numbers = []
    tmp_user_names = []
    tmp_user_ratings = []
    tmp_user_reviews = []

    # 후기 클릭-----------------------------------------------------------------------------------------------
    for key in tag_dict.keys():

        driver.find_element(By.ID, tag_dict[key]).click()
        time.sleep(1)
        # 후기 리뷰 개수
        search_review_num = re.compile('\d+[,]?\d+')
        try:

            tmp_review_count = int(search_review_num.findall(driver.find_element(By.ID, tag_dict[key]).text)[0])
        except:
            try:

                tmp_review_count = int(search_review_num.findall(driver.find_element(By.ID, tag_dict[key]).text)[0].replace(',', ''))
            except:
                search_review_num = re.compile('\d+')
                tmp_review_count = int(search_review_num.findall(driver.find_element(By.ID, tag_dict[key]).text)[0])

        if tmp_review_count == 0:
            print(f'{key} 없음')
            # 다음 리뷰 크롤링으로 이동

        else:
            # 리뷰 1페이지 크롤링
            req = driver.page_source
            soup = BeautifulSoup(req, 'html.parser')

            tmp_names1, tmp_numbers1, tmp_user_names1, tmp_user_ratings1, tmp_user_reviews1 = review_crawling(soup)
            # print(f'{key} 1페이지 유저 이름: {tmp_user_names1}')

            tmp_names = tmp_names + tmp_names1
            tmp_numbers = tmp_numbers + tmp_numbers1
            tmp_user_names = tmp_user_names + tmp_user_names1
            tmp_user_ratings = tmp_user_ratings + tmp_user_ratings1
            tmp_user_reviews = tmp_user_reviews + tmp_user_reviews1

            # 리뷰가 10개 초과여서 다음 페이지 크롤링이 필요한 경우
            if tmp_review_count > 10:
                # 후기 다음 페이지 클릭
                # 다른 요소로 스크롤하여 클릭하려는 요소가 보이도록 만들기
                # from selenium.webdriver.common.keys import Keys

                try:
                    target_element = driver.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[4]')   #2페이지
                    driver.execute_script("arguments[0].scrollIntoView(true);", target_element)
                    target_element.click()

                except:
                    print("2페이지로 넘어갈 수 없습니다.")

                # time.sleep(1)
                # driver.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[4]').click()
                time.sleep(1)

                req = driver.page_source
                soup = BeautifulSoup(req, 'html.parser')

                tmp_names2, tmp_numbers2, tmp_user_names2, tmp_user_ratings2, tmp_user_reviews2 = review_crawling(soup)
                # print(f'{key} 2페이지 유저 이름: {tmp_user_names2}')

                tmp_names = tmp_names + tmp_names2[:5]
                tmp_numbers = tmp_numbers + tmp_numbers2[:5]
                tmp_user_names = tmp_user_names + tmp_user_names2[:5]
                tmp_user_ratings = tmp_user_ratings + tmp_user_ratings2[:5]
                tmp_user_reviews = tmp_user_reviews + tmp_user_reviews2[:5]
                # print(f'{key} 수집 후 유저 이름: {tmp_user_names}')


            list_len = 15 if tmp_review_count >= 15 else tmp_review_count
            print(f'{key} 수집: {list_len} 개')

    return tmp_names, tmp_numbers, tmp_user_names, tmp_user_ratings, tmp_user_reviews



---



## 자동화

In [87]:
# 실제로는 100페이지 정도를 추출해서 각 페이지 당 10개씩 추출하는 등으로 진행
page = sorted(rd.sample(range(26, 27), 1))
page

[26]

In [97]:
import random as rd

# 페이지 내에 상품 랜덤 추출
random_product = rd.sample(range(82, 91), 8)
random_product = sorted(random_product)
random_product

[82, 83, 84, 85, 86, 87, 88, 89]

In [98]:
import time
# 상의 페이지(1 page)

major_category = [] # 대분류
middle_category = [] # 중분류
names = [] # 제품명
numbers = [] # 품번
limit = [] # 한정판매
exclusive = [] # 단독판매
target_gen = [] # 타겟성별
buy_gen = [] # 구매 성별
buy_age = [] # 구매 나이
month_view = [] # 1개월 조회수
like_cnt = [] # 좋아요 수
pro_price = [] # 판매 가격
discount_rate=[] # 할인률
delivery_date = [] # 배송일자
pro_rating = [] # 평점
review = [] # 리뷰 수
buy = [] # 1년 간 누적 판매

df_musinsa_review_raw = pd.DataFrame(columns=['product_names', 'product_ids', 'user_names', 'user_ratings', 'user_reviews'])   #리뷰 수집df
tag_dict = {'스타일 후기': 'estimate_style', '상품 후기': 'estimate_photo', '일반 후기': 'estimate_goods'}

driver = webdriver.Chrome()
for i in page:
    # 페이지 열기
    driver.get("https://www.musinsa.com/categories/item/002?d_cat_cd=001&brand=&list_kind=small&sort=pop_category&sub_sort=&page={}&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=".format(i))
    driver.maximize_window()
    time.sleep(3)

    for j in random_product:
        print(f'{i}페이지 {j}번 상품 리뷰 크롤링 시작-------------------')

        try:
        # 제품 클릭하기 여기서 화면 크기때문에 클릭이 안될 수 있음. if문으로 스크롤 넣어주기
            first_content = driver.find_element(By.CSS_SELECTOR, '#searchList > li:nth-child({}) > div.li_inner > div.article_info > p.list_info > a'.format(j))
            first_content.click()
            time.sleep(2)

        except:
            driver.get("https://www.musinsa.com/categories/item/002?d_cat_cd=001&brand=&list_kind=small&sort=pop_category&sub_sort=&page={}&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=".format(i))
            first_content = driver.find_element(By.CSS_SELECTOR, '#searchList > li:nth-child({}) > div.li_inner > div.article_info > p.list_info > a'.format(j+1))
            first_content.click()
            time.sleep(2)

        # 화면 html 코드 가져오기
        req = driver.page_source
        soup = BeautifulSoup(req, 'html.parser')

        

        # 대분류와 중분류
        
        # # out of list error 발생 시 time.sleep()을 걸어보세요.
        # category1 = soup.select_one('div.product-detail__sc-up77yl-0.htaeEr').get_text().split('\xa0')[0]
        # major_category.append(category1)

        try:
            category1 = soup.select_one('div.product-detail__sc-up77yl-0.htaeEr').get_text().split('\xa0')[0]
            major_category.append(category1)
        except:
            driver.refresh()
            time.sleep(2)
            req = driver.page_source
            soup = BeautifulSoup(req, 'html.parser')
            category1 = soup.select_one('div.product-detail__sc-up77yl-0.htaeEr').get_text().split('\xa0')[0]
            major_category.append(category1)

        category2 = soup.select_one('div.product-detail__sc-up77yl-0.htaeEr').get_text().split('\xa0')[2]
        middle_category.append(category2)

        # 이름
        name = soup.select_one('h3.product-detail__sc-1klhlce-3.fjguJZ').get_text()
        names.append(name)

        # 품번
        numbers.append(품번())

        # 한정판매
        limit.append(한정_단독()[0])

        # 단독판매
        exclusive.append(한정_단독()[1])

        # 타겟 젠더
        target_gen.append(성별2())

        # 구매 성별
        buy_gen.append(구매성별())

        # 구매 나이
        buy_age.append(구매나이())

        # 1개월 조회수
        try:
            month_view.append(조회수2())
        except:
            month_view.append(조회수())

        # 좋아요 수
        like_cnt.append(좋아요())

        # 가격(가격)
        pro_price.append(가격())

        # 할인율
        discount_rate.append(할인율())

        # 배송일자
        delivery_date.append(배송일자())

        # 평점
        pro_rating.append(평점())

        # 후기수
        review.append(후기수())

        # 구매수
        buy.append(누적판매())

        #리뷰 크롤링
        product_names, product_ids, user_names, user_ratings, user_reviews = get_tmp_df_musinsa_review_raw(driver, tag_dict)

        tmp_df_musinsa_review_raw = pd.DataFrame({'product_names': product_names, 'product_ids': product_ids, 'user_names': user_names, 'user_ratings': user_ratings, 'user_reviews': user_reviews})

        df_musinsa_review_raw = pd.concat([df_musinsa_review_raw, tmp_df_musinsa_review_raw], axis=0)
        print(f'Total DF rows: {df_musinsa_review_raw.count()[1]}')
        driver.back()
        time.sleep(1)

26페이지 82번 상품 리뷰 크롤링 시작-------------------
스타일 후기 수집: 1 개
상품 후기 수집: 2 개
일반 후기 수집: 3 개
Total DF rows: 6


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_29524\2521031417.py:128: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'Total DF rows: {df_musinsa_review_raw.count()[1]}')


26페이지 83번 상품 리뷰 크롤링 시작-------------------
스타일 후기 없음
상품 후기 없음
일반 후기 수집: 1 개
Total DF rows: 7


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_29524\2521031417.py:128: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'Total DF rows: {df_musinsa_review_raw.count()[1]}')


26페이지 84번 상품 리뷰 크롤링 시작-------------------
스타일 후기 없음
상품 후기 없음
일반 후기 없음
Total DF rows: 7


...

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_29524\2521031417.py:128: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'Total DF rows: {df_musinsa_review_raw.count()[1]}')


26페이지 88번 상품 리뷰 크롤링 시작-------------------
2페이지로 넘어갈 수 없습니다.
스타일 후기 수집: 15 개
2페이지로 넘어갈 수 없습니다.
상품 후기 수집: 15 개
2페이지로 넘어갈 수 없습니다.
일반 후기 수집: 15 개
Total DF rows: 109


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_29524\2521031417.py:128: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'Total DF rows: {df_musinsa_review_raw.count()[1]}')


26페이지 89번 상품 리뷰 크롤링 시작-------------------
스타일 후기 없음
상품 후기 수집: 1 개
일반 후기 없음
Total DF rows: 110


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_29524\2521031417.py:128: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'Total DF rows: {df_musinsa_review_raw.count()[1]}')


In [99]:
len(major_category),len(middle_category),len(names), len(numbers), len(limit), len(exclusive), len(target_gen), len(buy_gen), len(month_view), len(like_cnt), len(pro_price),len(discount_rate) ,len(delivery_date), len(pro_rating), len(review), len(buy)

(8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8)

---

In [108]:
# df_4 = pd.read_csv('[이청하]musinsa_아우터_240119_4.csv')
len(df_4)

4

In [120]:
# df_5 = pd.read_csv('D:/0_zerobase/프로젝트/2.크롤링한 파일 모음/sv 폴더 모음_내꺼_01.19/[이청하]musinsa_아우터_240119_5.csv')
len(df_5)

7

In [121]:
df=[]
df = pd.concat([df_4, df_5], axis=0, ignore_index=True)
len(df)

11

In [122]:
len(df_6)

149

In [123]:
df = pd.concat([df, df_6], axis=0, ignore_index=True)
len(df)

160

In [124]:
len(df_6_2)

11

In [125]:
df = pd.concat([df, df_6_2], axis=0, ignore_index=True)
len(df)

171

In [126]:
len(df_6_3)

8

In [128]:
df = pd.concat([df, df_6_3], axis=0, ignore_index=True)
len(df)

179

In [129]:
len(df_7)

270

In [130]:
df = pd.concat([df, df_7], axis=0, ignore_index=True)
len(df)

449

In [131]:
df.to_csv('[이청하]musinsa_아우터_240119.csv', index=False)

---

In [135]:
# df_reivew_4 = pd.read_csv('[이청하]review_아우터_240119.csv_4')
len(df_musinsa_review_raw_4)

38

In [136]:
# df_reivew_5 = pd.read_csv('[이청하]review_아우터_240118.csv_5')
len(df_musinsa_review_raw_5)

184

In [137]:
df_review = pd.concat([df_musinsa_review_raw_4, df_musinsa_review_raw_5], axis=0, ignore_index=True)
len(df_review)

222

In [138]:
# df_musinsa_review_raw_6
len(df_musinsa_review_raw_6)

3206

In [139]:
df_review = pd.concat([df_review, df_musinsa_review_raw_6], axis=0, ignore_index=True)
len(df_review)

3428

In [141]:
len(df_musinsa_review_raw_6_2)

146

In [142]:
df_review = pd.concat([df_review, df_musinsa_review_raw_6_2], axis=0, ignore_index=True)
len(df_review)

3574

In [143]:
len(df_musinsa_review_raw_6_3)

110

In [144]:
df_review = pd.concat([df_review, df_musinsa_review_raw_6_3], axis=0, ignore_index=True)
len(df_review)

3684

In [145]:
len(df_musinsa_review_raw_7)

5579

In [146]:
df_review = pd.concat([df_review, df_musinsa_review_raw_7], axis=0, ignore_index=True)
len(df_review)

9263

In [147]:
df_review.to_csv('[이청하]review_아우터_240119.csv', index=False)

# df_6_3

In [100]:
import pandas as pd

df_6_3 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [101]:
df_6_3.to_csv('[이청하]musinsa_아우터_240119_6_3.csv', index=False)

In [102]:
df_musinsa_review_raw_6_3 = df_musinsa_review_raw
df_musinsa_review_raw_6_3

,product_names,product_ids,user_names,user_ratings,user_reviews
0,브리티시 울 헤링본 재킷,LE4FJKW04BR,leanie_minie,100,색도 예쁘고 옷이 가볍고 따뜻하구 보들보들해서 움직이기도 편해여 포장이 멋져요
1,브리티시 울 헤링본 재킷,LE4FJKW04BR,참도장깨기,100,m사이즈라 그런지 크진 않아요 어깨 좁은편인데 보완돼서 좋아요
2,브리티시 울 헤링본 재킷,LE4FJKW04BR,leanie_minie,100,색도 예쁘고 옷이 가볍고 따뜻하구 보들보들해서 움직이기도 편해여 포장이 멋져요
3,브리티시 울 헤링본 재킷,LE4FJKW04BR,어딕,100,작년부터 눈 여겨봤던 자켓인데 세일할 때 너무 잘 샀어요 품질도 좋고 핏도 디자인도...
4,브리티시 울 헤링본 재킷,LE4FJKW04BR,참도장깨기,100,m사이즈라 그런지 크진 않아요 어깨 좁은편인데 보완돼서 좋아요
...,...,...,...,...,...
41,오버사이즈 칼라드 보머 재킷 [더스티 베이지],MMAUJ704-UI,뉴비_871bb64d,80,선물용으로 사이즈 고민했는데 잘 맞아요평소 넉넉하게 105 입는데 라지 좋습니다
42,오버사이즈 칼라드 보머 재킷 [더스티 베이지],MMAUJ704-UI,hemaljisu,100,좋습니다 아주 다른색깔도 추가 구매할 예정입니다.
43,오버사이즈 칼라드 보머 재킷 [더스티 베이지],MMAUJ704-UI,제이래브,100,옷 이쁘네요. ma-1 같은데 카라깃이 있어서 특이해요.
44,오버사이즈 칼라드 보머 재킷 [더스티 베이지],MMAUJ704-UI,재이재이,100,좋은 제품 감사합니다 선생님 안녕하세요 여러분 추천


In [103]:
df_musinsa_review_raw_6_3.to_csv('[이청하]review_아우터_240119.csv_6_3', index=False)

# df_6_2

In [92]:
import pandas as pd

df_6_2 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [93]:
df_6_2.to_csv('[이청하]musinsa_아우터_240119_6_2.csv', index=False)

In [94]:
df_musinsa_review_raw_6_2 = df_musinsa_review_raw
df_musinsa_review_raw_6_2

,product_names,product_ids,user_names,user_ratings,user_reviews
0,Crochet Knit Jacket Orange,TN223KKNJK01OR,Kwhiln,100,정말 실물 예쁘네요. 재질도 엄청 좋습니다. 특이하지만 이상하지 않고 잘 입고 다닐...
1,Crochet Knit Jacket Orange,TN223KKNJK01OR,마웅융,100,크로쉐가 주는 빈티지하고 귀여운 느낌과 디네댓의 감성이 조화로워 독특한 느낌을 줍니...
2,Crochet Knit Jacket Orange,TN223KKNJK01OR,마웅융,100,크로쉐가 주는 귀여운 느낌과 디네댓의 감성이 조화로워 독특한 느낌을 줍니다. 소매가...
0,[사계절] 커스터마이징 세미오버 핏 수트 블레이저 (5컬러),5000196047,웁둥,100,포장도 꼼꼼하고 스타일도 좋아지고 소개팅 할때 입으면 좋을 것 같아요~
1,[사계절] 커스터마이징 세미오버 핏 수트 블레이저 (5컬러),5000196047,나뭇잎 마을의 몽키 D 루피,100,생각보다 색이 더 어두웠는 느낌도 있고 재질도 살짝 까끌까끌했지만 핏은 마음에 듭니다!
...,...,...,...,...,...
35,니트 투웨이 집업,FRK-1263,park9912,60,전체적으로 그냥 다 평범한 느낌인데 그냥 잠깐 외출이나 편의점 슈퍼 등 갈 따 잊으...
36,니트 투웨이 집업,FRK-1263,Pubgwat,100,기장이 살짝 길어 아쉽긴한데 이쁘게 입기 좋은 옷
37,니트 투웨이 집업,FRK-1263,갓영구,80,생각했던 것보다는 얇아요지퍼도 조금 빡빡 그래도 디자인은 마음에 들어요
38,니트 투웨이 집업,FRK-1263,뉴비_76309c34da19,100,프리사이즈라 부담 없이 잘 입고 다녀요 가을 봄에 입기 딱좋은


In [95]:
df_musinsa_review_raw_6_2.to_csv('[이청하]review_아우터_240119.csv_6_2', index=False)

# 추가 수집 - df_7

In [81]:
import pandas as pd

df_7 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [86]:
df_7.to_csv('[이청하]musinsa_아우터_240119_7.csv', index=False)

In [83]:
df_musinsa_review_raw_7 = df_musinsa_review_raw
df_musinsa_review_raw_7

,product_names,product_ids,user_names,user_ratings,user_reviews
0,유틸리티 자켓 블랙,유틸리티 자켓 블랙,AG Club,100,지금 입기엔 좀 얇고 날 풀리면 입기 좋을 듯 해요밑에 끈 쪼여서 입으면 진짜 예뻐요
1,유틸리티 자켓 블랙,유틸리티 자켓 블랙,굥굥굥굥굥굥,100,얇아서 봄에 입기 너무 좋습니다 후드티 맨투맨보단 니트나 셔츠 긴팔티를 이너로 하는...
2,유틸리티 자켓 블랙,유틸리티 자켓 블랙,z1_y0ung,100,물량폭주로 주문지연되서 거의 2주만에 받아서 짜증났는데 상품 택도 안달려있어서 2차...
3,유틸리티 자켓 블랙,유틸리티 자켓 블랙,구름이랑젤리,100,흠 봄에 잘 입었어요 사이즈가 좀 더 커도 좋을것같다만 예뻐요
4,유틸리티 자켓 블랙,유틸리티 자켓 블랙,eunruel,100,생각없이 샀는데 평소에 되게 많이 착용합니다!! 엄청 편하고 재질도 편하고 그냥 다...
...,...,...,...,...,...
10,CHEST DOUBLE POCKET COAT_BEIGE,20BAFWOCTU001BE,Younida,100,좋은 상품 구매 잘얻고 갑니다 마음에 드네요 항상 값진 결과물 입니다
11,CHEST DOUBLE POCKET COAT_BEIGE,20BAFWOCTU001BE,뉴비_7aa4bef3f639,100,오버핏으려고 입으려고 사긴했는데 예상보다 좀 더 크긴 한데 가격대비 정말 좋아요
12,CHEST DOUBLE POCKET COAT_BEIGE,20BAFWOCTU001BE,제주마스타,100,"배드인배드 제품 여러가지 구매중인뎁,,코트 스타일 좋아요좋아 ~어떠한 바지와도 무..."
13,CHEST DOUBLE POCKET COAT_BEIGE,20BAFWOCTU001BE,뉴비_489c19df34cb,100,진짜 제가 완전 자연산 멸치인데 핏 좋네요ㅋㅋ개이득


In [84]:
df_musinsa_review_raw_7.to_csv('[이청하]review_아우터_240119.csv_7', index=False)

---

# 추가 수집 - df_6

In [70]:
import pandas as pd

df_6 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [71]:
df_6.to_csv('[이청하]musinsa_아우터_240119_6.csv', index=False)

In [72]:
df_musinsa_review_raw_6 = df_musinsa_review_raw
df_musinsa_review_raw_6

,product_names,product_ids,user_names,user_ratings,user_reviews
0,우먼즈 덕다운 숏 패딩 푸퍼 아이보리,ASBPJP004_3,특수이모티콘,100,"패딩이 너무너무 따슈워용!!비록 팔이 너무 길었지만,보온성은 너무너무 좋았답니다"
1,우먼즈 덕다운 숏 패딩 푸퍼 아이보리,ASBPJP004_3,hju__u,100,똑딱이 단추 부분이 약해서 안잠그고 다니려구요패딩은 너무 귀엽고 예쁩니당
2,우먼즈 덕다운 숏 패딩 푸퍼 아이보리,ASBPJP004_3,당근쏨쏨,100,합리적인 가격에 덕다운 구매했어요!! 기장을 걱정했는데 기장감 적당하고 가볍고 편해...
3,우먼즈 덕다운 숏 패딩 푸퍼 아이보리,ASBPJP004_3,김박현숙,80,처음봤을때 생각보다 더 노란끼가 있어서 아쉬웠는데 계속 보다보니 또 적응돼서 괜찮은...
4,우먼즈 덕다운 숏 패딩 푸퍼 아이보리,ASBPJP004_3,특수이모티콘,100,"패딩이 너무너무 따슈워용!!비록 팔이 너무 길었지만,보온성은 너무너무 좋았답니다"
...,...,...,...,...,...
21,코위찬 니트 집업 바시티_네이비,CBCFUKC008NY,뉴비_452d17e5f34d,100,특별한 디자인 니트짚업 마음에들어요.잘산것같아요
22,코위찬 니트 집업 바시티_네이비,CBCFUKC008NY,두랄루돈,100,M 구매했다가 조금 애매해서 L로 바꿨는데 L가 딱이네요가재가 너무 귀여워서 홀린듯...
23,코위찬 니트 집업 바시티_네이비,CBCFUKC008NY,댜._.댜,100,브라운이랑 고민하다가 네이비 샀는데 넘 맘에 들어요! 초겨울까지 단독으로 입기 좋을...
24,코위찬 니트 집업 바시티_네이비,CBCFUKC008NY,뉴비_c65b337c,80,아직 입기에는 두꺼워서 잘 보관중~청바지에 걸쳐입으면 이뽀 이뽀요^^잘입을께요.


In [73]:
df_musinsa_review_raw_6.to_csv('[이청하]review_아우터_240119.csv_6', index=False)

---

# 추가 수집 - df_5

In [57]:
import pandas as pd

df_5 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [58]:
df_5

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
0,아우터,스타디움 재킷,[NF] 스키퍼 토글 자켓 (GRAY)_F23ZA937,F23ZA937GR,1,0,1,1.0,"(19~23세, 0)",5900.0,2651,129500.0,30,5,4.9,37,100.0
1,아우터,스타디움 재킷,레글런 오버핏 배색 버튼 바시티 자켓 점퍼 [네이비],23F88212-4100,0,0,1,1.0,"(19~23세, 0)",4000.0,1368,66780.0,58,10,4.7,18,100.0
2,아우터,트레이닝 재킷,Reversible Team Jacket Grey,TN223WOWLS01GR,0,0,2,1.0,"(19~23세, 24~28세)",3400.0,3905,89500.0,50,6,5.0,68,150.0
3,아우터,겨울 더블 코트,브라운 오버사이즈 로브 코트 (PDHAX7018),PDHAX7018,0,0,0,0.0,"(29~33세, 0)",6600.0,473,119000.0,83,5,4.9,22,100.0
4,아우터,숏패딩/숏헤비 아우터,우먼즈 덕다운 숏 패딩 푸퍼 아이보리,ASBPJP004_3,0,0,1,NaN,"(29~33세, 0)",2300.0,636,155400.0,40,5,4.8,13,NaN
5,아우터,트레이닝 재킷,[SS Ver.]파이핑 오버핏 스웨트 집업_Black,m1037_685405,0,0,2,1.0,"(~18세, 0)",900.0,2989,22800.0,54,6,4.7,44,250.0
6,아우터,나일론/코치 재킷,[US] Champion 로고 스타디움 풀집 자켓 (NORMAL NAVY),CKJA3E010N2,1,0,0,0.0,"(24~28세, 0)",1500.0,3600,71400.0,40,6,4.8,86,350.0


In [59]:
df_5.to_csv('[이청하]musinsa_아우터_240118_5.csv', index=False)

In [60]:
df_musinsa_review_raw_5 = df_musinsa_review_raw
df_musinsa_review_raw_5

,product_names,product_ids,user_names,user_ratings,user_reviews
0,[NF] 스키퍼 토글 자켓 (GRAY)_F23ZA937,F23ZA937GR,Herohyun,100,예뻐영 날씨 좀만 괜찮아지명 입으려구용 지금은 추움
1,[NF] 스키퍼 토글 자켓 (GRAY)_F23ZA937,F23ZA937GR,kwonsy,100,굿 완전 강추 합니다!!! 투웨이라 아주 편하고 지퍼 끝까지 올려 입어도 이쁘네요 ...
2,[NF] 스키퍼 토글 자켓 (GRAY)_F23ZA937,F23ZA937GR,호일야,100,상품 사진 봤을 때 그렇게 널널하지 않을 거라는 예상했습니당다른 분들한테는 괜찮은 ...
3,[NF] 스키퍼 토글 자켓 (GRAY)_F23ZA937,F23ZA937GR,냥머앙,80,지금 완전 잘 입고 있어요 너무 이쁘고 맘애 들어요 어디든지 다 잘 어울려요
4,[NF] 스키퍼 토글 자켓 (GRAY)_F23ZA937,F23ZA937GR,한개미ㅇ,100,생각보다 되게 따뜻하고 포닥포닥하네요세일할 때 잘 산 것 같아요떡볶이 코트 겸 추리...
...,...,...,...,...,...
39,[US] Champion 로고 스타디움 풀집 자켓 (NORMAL NAVY),CKJA3E010N2,김형린,100,좋아요 마음에 들어요 다음에 또 사려고 생각중이에요
40,[US] Champion 로고 스타디움 풀집 자켓 (NORMAL NAVY),CKJA3E010N2,그도그도,100,스포티한 바람막이를 좋아해서 샀는데 이쁘고 좋아요
41,[US] Champion 로고 스타디움 풀집 자켓 (NORMAL NAVY),CKJA3E010N2,농재,100,"원하는 핏이라 만족하고, 기장이 조금은 긴 건 참고해야할 것 같아요"
42,[US] Champion 로고 스타디움 풀집 자켓 (NORMAL NAVY),CKJA3E010N2,Ryani,100,"바람막이 러버인데, 챔피온 브랜드 이름에 맞게 퀄리티가 좋은거 같아요"


In [61]:
df_musinsa_review_raw_5.to_csv('[이청하]review_아우터_240118.csv_5', index=False)

---

# 추가 수집

In [48]:
import pandas as pd

df_4 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [49]:
df_4

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
0,아우터,겨울 싱글 코트,브라운 맥시 체스터 코트 (PDHAX7006),PDHAX7006,0,0,0,NaN,"(29~33세, 0)",3000.0,143,99900.0,86,5,5.0,1,NaN
1,아우터,카디건,THREE RIBBON CARDIGAN,IBC23WCD05BK,0,0,1,1.0,"(19~23세, 0)",4700.0,2559,58000.0,0,6,5.0,9,50.0
2,아우터,스타디움 재킷,[NF] 스키퍼 토글 자켓 (GRAY)_F23ZA937,F23ZA937GR,1,0,1,1.0,"(19~23세, 0)",5900.0,2651,129500.0,30,5,4.9,37,100.0
3,아우터,스타디움 재킷,레글런 오버핏 배색 버튼 바시티 자켓 점퍼 [네이비],23F88212-4100,0,0,1,1.0,"(19~23세, 0)",3900.0,1368,66780.0,58,10,4.7,18,100.0


In [50]:
df_4.to_csv('[이청하]musinsa_아우터_240118_4.csv', index=False)

In [51]:
df_musinsa_review_raw_4 = df_musinsa_review_raw
df_musinsa_review_raw_4

,product_names,product_ids,user_names,user_ratings,user_reviews
0,브라운 맥시 체스터 코트 (PDHAX7006),PDHAX7006,키위쥬스님,100,이쁘네요 잘입겠습니다 핏도 이쁘고 전체적으로 만족합니다. 색감도 제취향이에요 너무 ...
0,THREE RIBBON CARDIGAN,IBC23WCD05BK,ALPETA,100,리본에 잠식된 패션계...입으면 바디라인이 슬림해보이고 분홍 리본이 포인트가 되어줍니다~
1,THREE RIBBON CARDIGAN,IBC23WCD05BK,ALPETA,100,아직 입고 나가진못했지만 리본이 너무 귀여운 가디건이에용핏되는 착용감이 마음에 듭니다~
2,THREE RIBBON CARDIGAN,IBC23WCD05BK,멍멍ㄴ멈머,100,디자인이 넘 취향이라 구매했는데 생각했던거보다 핏도 너무 예쁘고 말라보임 핑크 리본...
3,THREE RIBBON CARDIGAN,IBC23WCD05BK,ALPETA,100,여름에 리본 슬리브리스 구매할까 말까~ 고민했었는데 FW시즌에 너무 귀여운 가디건이...
4,THREE RIBBON CARDIGAN,IBC23WCD05BK,L.H.j.,100,발레코어st 가디건 찾고 있었는데 딱 원하는 디자인이라 바로 구매했어요 얇은데 라인...
5,THREE RIBBON CARDIGAN,IBC23WCD05BK,박력너구리,100,"상세 사이즈보고이 옷 되게 작겠다,, 싶었음그러나 리본 너무 기여워서 못 참고 구매..."
6,THREE RIBBON CARDIGAN,IBC23WCD05BK,커피한잔하실래여?,100,친구에게 선물로 줬누ㅡㄴ데 친구가 좋아해 줬어요 예뻐요!!
7,THREE RIBBON CARDIGAN,IBC23WCD05BK,헿흐,100,하아.. 진짜 예쁘게 말라보이는 핏 너무 맘에 들어욧..
8,THREE RIBBON CARDIGAN,IBC23WCD05BK,멍멍ㄴ멈머,100,생각했던거보다 핏도 너무 예쁘고 말라보임 핑크 리본 디테일듀 너무 취저에요


In [52]:
df_musinsa_review_raw_4.to_csv('[이청하]review_아우터_240118.csv_4', index=False)

---

# 무신사 상품 크롤링 결과

In [98]:
import pandas as pd

df_2_3 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [99]:
df_2_3

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
0,아우터,숏패딩/숏헤비 아우터,MOLDY DYED PUFFER JACKET BLACK,FDRWCUA101M,0,0,2,0,"(24~28세, 0)",3400.0,634,296100.0,10,4,4.8,18,50.0
1,아우터,숏패딩/숏헤비 아우터,072903 경량 바머 자켓,5000402643,0,0,0,0,"(24~28세, 0)",650.0,188,58800.0,63,4,4.7,7,50.0


In [100]:
df_2_3.to_csv('[이청하]musinsa_아우터_240118_2_3.csv', index=False)

---

In [89]:
import pandas as pd

df_2_2 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [90]:
df_2_2

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
0,아우터,블루종/MA-1,Flight Jumper Olive,RS2323151,0,0,0,NaN,"(24~28세, 0)",1100.0,171,368000.0,0,4,5.0,2,NaN
1,아우터,겨울 기타 코트,세인트 오버핏 프리미엄 울 더플코트 [NAVY],DFCT2409NV,1,0,0,0.0,"(29~33세, 0)",2700.0,1914,254400.0,20,4,4.8,336,50.0
2,아우터,숏패딩/숏헤비 아우터,하이넥 아웃포켓 구스 다운 [라이트그레이],TMPD012140LGR,0,0,0,0.0,"(40세~, 0)",3700.0,540,59000.0,75,5,4.7,44,50.0
3,아우터,스타디움 재킷,VELVET VARSITY JACKET BLACK,RS-V02,0,0,2,0.0,"(19~23세, 0)",2900.0,9969,153300.0,30,4,4.9,1293,800.0
4,아우터,롱패딩/롱헤비 아우터,FW23 UNISEX PADDED BALLOON LONG COAT #BLACK,312321,0,0,1,NaN,"(~18세, 0)",600.0,103,320000.0,0,5,NaN,0,NaN
5,아우터,롱패딩/롱헤비 아우터,여성 퀼팅 립스탑 라이닝 W QUILTED RIPSTOP LINIG,WO136QJJPZ07IV,0,0,1,1.0,"(40세~, 0)",3000.0,512,59000.0,63,5,4.8,8,50.0
6,스포츠/용품,아우터,5219JS238 남성 망사 등판 초경량 방풍 자켓,5219JS238BL,0,0,0,NaN,"(0, 0)",500.0,10,14900.0,89,4,NaN,0,NaN
7,아우터,스타디움 재킷,남성 새틴 테디 재킷 - 블랙 / KM02233WQ0052P198,KM02233WQ0052P198SS23,0,0,0,0.0,"(29~33세, 0)",1500.0,487,397000.0,42,4,4.9,19,50.0
8,아우터,후드 집업,남성 클래식 버니 3 집업 후드 - 차콜 멜란지 / M32MS600210,M32MS600210FW23,0,0,0,0.0,"(40세~, 0)",2200.0,175,373000.0,41,4,5.0,13,50.0
9,아우터,베스트,케이블 니트 코튼 V넥 스웨터 베스트 - 화이트,WMPOSWENC020681-100,0,0,1,1.0,"(29~33세, 0)",1800.0,2024,199000.0,0,4,4.9,64,150.0


In [91]:
df_2_2.to_csv('[이청하]musinsa_아우터_240118_2_2.csv', index=False)

---

In [70]:
import pandas as pd

df_1_3 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [71]:
df_1_3

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
0,아우터,기타 아우터,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,0,1,0,0.0,"(40세~, 0)",1600.0,2151,52390.0,25,3,4.8,1738,500.0
1,아우터,숏패딩/숏헤비 아우터,덱 패딩 자켓(카키),NPD0062KHZ,1,0,0,0.0,"(29~33세, 0)",6700.0,784,56700.0,70,5,4.8,58,200.0
2,아우터,스타디움 재킷,R 시티 헤비 바시티자켓 (딥 그린),RWJK22003GR,0,0,0,0.0,"(40세~, 0)",5000.0,5558,89600.0,30,4,4.8,168,700.0
3,아우터,카디건,울 부클 카라 가디건 니트 (IVORY),TRFKN06IV,1,0,2,0.0,"(19~23세, 0)",3800.0,912,43050.0,27,5,4.8,70,200.0
4,아우터,나일론/코치 재킷,T-Light Jacket Black,TN230WOWLS02BK,0,0,2,1.0,"(19~23세, 0)",4400.0,4373,65400.0,40,5,4.9,308,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,아우터,숏패딩/숏헤비 아우터,리버서블 덤블 숏푸퍼 WHJPD1192U,WHJPD1192U,1,0,0,1.0,"(19~23세, 0)",8000.0,19633,129000.0,0,4,4.9,3524,1300.0
70,스포츠/용품,아우터,NJ1DQ55A 남성 1996 에코 눕시 자켓,NJ1DQ55A,1,0,0,0.0,"(24~28세, 40세~)",128000.0,44706,399000.0,0,5,4.9,4939,6200.0
71,아우터,후드 집업,FLOWER ZIP UP HOOD(BROWN),HZALZBA,0,0,1,1.0,"(19~23세, 0)",1600.0,1067,89000.0,0,4,5.0,35,100.0
72,아우터,플리스/뽀글이,Y.E.S Boa Fleece Bonded Jacket Black,YES598,0,0,2,1.0,"(19~23세, 0)",5600.0,1954,63600.0,60,5,4.8,96,100.0


In [72]:
df_1_3.to_csv('[이청하]musinsa_아우터_240118_1_3.csv', index=False)

---

In [59]:
import pandas as pd

df_1_2 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [60]:
df_1_2

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
0,아우터,블루종/MA-1,비건 스웨이드 오버핏 미니멀 인시전 스트레이트 집업 자켓 [블랙],B5,1,0,2,0.0,"(19~23세, 0)",49000.0,5817,39900.0,71,5,4.8,287,1800.0
1,아우터,카디건,Minimal Crop Sleeveless Set - 4COL,23SS02,0,0,1,1.0,"(24~28세, 0)",11000.0,7959,39750.0,25,60,4.5,144,700.0
2,아우터,카디건,풀오버 터틀넥 투웨이 케이블 카라 니트 집업 [아이보리],23FK222-1300,1,0,1,1.0,"(19~23세, 0)",5100.0,25473,45000.0,50,10,4.6,1028,700.0
3,아우터,패딩 베스트,(VLAD) 2WAY 어센틱 패딩 베스트_베이지,VBCOIV01BE,0,1,2,NaN,"(~18세, 0)",1800.0,220,0.0,34,5,5.0,1,NaN
4,아우터,숏패딩/숏헤비 아우터,하이넥 헤비 덕다운 숏패딩 크림,HD23FWJK01,0,0,1,1.0,"(19~23세, 0)",2900.0,928,125400.0,40,5,5.0,5,50.0
5,아우터,기타 아우터,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,0,1,0,0.0,"(40세~, 0)",1600.0,2151,52390.0,25,3,4.8,1738,500.0


In [61]:
df_1_2.to_csv('[이청하]musinsa_아우터_240118_1_2.csv', index=False)

---

In [49]:
import pandas as pd

df_1 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [50]:
df_1

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
0,아우터,플리스/뽀글이,베이직 플리스 자켓 (U00DTJK59),2646018,0,0,0,1.0,"(~18세, 0)",6300.0,9307,19900.0,53,4,4.7,1092,2800.0
1,아우터,숏패딩/숏헤비 아우터,베이직 푸퍼_SPJPE11C11,SPJPE11C11,1,0,0,0.0,"(19~23세, 0)",560000.0,127420,62910.0,10,5,4.9,42691,76000.0
2,아우터,플리스/뽀글이,베이직 퍼플리스 집업_SPFZD4TU01,SPFZD4TU01,1,0,0,1.0,"(~18세, 0)",97000.0,69998,19900.0,33,5,4.8,26197,66000.0
3,아우터,숏패딩/숏헤비 아우터,파이핑 덕 다운 패딩_블랙_OL255,OL255,0,0,2,0.0,"(24~28세, 0)",27000.0,2639,54900.0,77,4,4.7,73,400.0
4,아우터,후드 집업,2WAY 스웻 후드 집업 (MELANGE GREY),TAF-2WHZU01MG,1,0,2,0.0,"(19~23세, 0)",96000.0,217723,39000.0,13,4,4.8,81354,10000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,아우터,후드 집업,[세트] 돌핀 자수 엣지 기모 후디 짚업 조거 트레이닝 팬츠 블랙,134522722BK,0,0,0,1.0,"(40세~, 0)",450.0,59,99800.0,50,4,5.0,9,50.0
96,아우터,패딩 베스트,여성 에어 플러스 R넥 구스다운 베스트_PHB5VP2012,PHB5VP2012,0,0,1,1.0,"(40세~, 0)",1600.0,104,14900.0,79,5,5.0,11,50.0
97,아우터,후드 집업,YOUHEE RIBBON HOOD ZIP-UP,YHHZDC001MG,0,0,1,1.0,"(24~28세, 0)",14000.0,7333,87200.0,20,4,4.8,534,1700.0
98,아우터,패딩 베스트,N234UDV950 칼리스 MODULA 경량 구스 다운 베스트 CARBON BLACK,N234UDV950/N234UDV050,0,0,0,1.0,"(40세~, 0)",3300.0,161,89000.0,10,4,4.8,25,100.0


In [51]:
df_1.to_csv('[이청하]musinsa_아우터_240118_1.csv', index=False)

---

In [41]:
import pandas as pd

df_3 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [42]:
df_3.to_csv('[이청하]musinsa_아우터_240118_3.csv', index=False)

In [32]:
import pandas as pd

df_2 = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [33]:
df_2.to_csv('[이청하]musinsa_아우터_240118_2.csv', index=False)

---

# 리뷰크로링 결과

In [101]:
df_musinsa_review_raw_2_3 = df_musinsa_review_raw
df_musinsa_review_raw_2_3

,product_names,product_ids,user_names,user_ratings,user_reviews
0,MOLDY DYED PUFFER JACKET BLACK,FDRWCUA101M,나는야쇼퍼홀릭,100,남자친구 입었는데퀄리티 좋네요이쁘고 저렴한 가격레 구매해서 좋아요
1,MOLDY DYED PUFFER JACKET BLACK,FDRWCUA101M,살바비어,80,176 65이고 S로 갔는데 숏패딩은 처음이라 아직도 어색하고 저한테 사이즈가 맞는...
2,MOLDY DYED PUFFER JACKET BLACK,FDRWCUA101M,돌잠금,100,기장감도 적당하고 어깨라인이 흘러내려와서 더 어깡으로 망들어주면서 두툼하니 겨울 잘...
3,MOLDY DYED PUFFER JACKET BLACK,FDRWCUA101M,녜녠녜,100,사이즈는 좀 크지만 벌키하게 입는 거 좋아해서 맘에 들어요 ! 은은한 크랙도 너무 ...
4,MOLDY DYED PUFFER JACKET BLACK,FDRWCUA101M,블루스공오,100,좋아여. 지퍼가 살짝 부드럽진 않은데 시간지나면 괜찮을 것 같아여.
5,MOLDY DYED PUFFER JACKET BLACK,FDRWCUA101M,나이젤카본ㅡ,100,포장도 꼼꼼하고 옷걸이까지 정말 만족스러운 배송이었습니다.은은한 크랙 디테일이 예쁘...
6,MOLDY DYED PUFFER JACKET BLACK,FDRWCUA101M,살바비어,80,176 65이고 S로 갔는데 숏패딩은 처음이라 아직도 어색하고 저한테 사이즈가 맞는...
7,MOLDY DYED PUFFER JACKET BLACK,FDRWCUA101M,녜녠녜,100,사이즈는 조금 크지만 핏이 넘 예뻐요 ! 따뜻하구 ... 수납공간 많아서 좋아용
8,MOLDY DYED PUFFER JACKET BLACK,FDRWCUA101M,사신무무,100,요거 첨봤을때부터 몬가 꽂힌상태로 숏패딩 계속 둘러보다가 결국 구매했는데 상세페이지...
9,MOLDY DYED PUFFER JACKET BLACK,FDRWCUA101M,블루스공오,100,좋아여. 지퍼가 살짝 부드럽진 않은데 시간지나면 괜찮을 것 같아여.


In [102]:
df_musinsa_review_raw_2_3.to_csv('[이청하]review_아우터_240118.csv_2_3', index=False)

In [92]:
df_musinsa_review_raw_2_2 = df_musinsa_review_raw
df_musinsa_review_raw_2_2

,product_names,product_ids,user_names,user_ratings,user_reviews
0,Flight Jumper Olive,RS2323151,wya_s,100,렆사 첨 구매하는데 퀄 핏 둘다 만족입니다. 올리브가 젤 이쁨
1,Flight Jumper Olive,RS2323151,wya_s,100,멋이 트질라이띠따이하고 깔끼하고 예쁠라이코스
0,세인트 오버핏 프리미엄 울 더플코트 [NAVY],DFCT2409NV,Hongik9,80,일단 코트이다보니까 쫌 불편하긴 한데 핏도 좋고 이뻐요
1,세인트 오버핏 프리미엄 울 더플코트 [NAVY],DFCT2409NV,__Seo.J,80,163에 여자예오 m사이즈 입었어요! 무거운데 그냥 울코트 무게입니당단추 끈이 약간...
2,세인트 오버핏 프리미엄 울 더플코트 [NAVY],DFCT2409NV,병아리뺙뺙,100,한겨울엔 못입지만 그래도 따뜻해요 부드럽진않아요
...,...,...,...,...,...
36,케이블 니트 코튼 V넥 스웨터 베스트 - 화이트,WMPOSWENC020681-100,Kangje,100,배송이 정말 빠르네요! 덕분에 가족행사에 잘 입었어요 사이즈도 넉넉하니 좋고 무엇보...
37,케이블 니트 코튼 V넥 스웨터 베스트 - 화이트,WMPOSWENC020681-100,중랑구면목,100,귀여워요 흰반팔에 입으면 찰떡 여유있게 편하게 잘맞아요
38,케이블 니트 코튼 V넥 스웨터 베스트 - 화이트,WMPOSWENC020681-100,뉴비_f2d2487299b3,100,사고 싶었던건데 이것저것 먹여서 젤 저렴하게 샀어요..화이트 반팔에 입는게 젤 깔끔...
39,케이블 니트 코튼 V넥 스웨터 베스트 - 화이트,WMPOSWENC020681-100,뉴비_8d873cc85c61,100,매번 눈여겨보고있었는데 조은가격으로 득템했습니다


In [93]:
df_musinsa_review_raw_2_2.to_csv('[이청하]review_아우터_240118.csv_2_2', index=False)

---

In [73]:
df_musinsa_review_raw_1_3 = df_musinsa_review_raw
df_musinsa_review_raw_1_3

,product_names,product_ids,user_names,user_ratings,user_reviews
0,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,skdlfjl,100,귀여운 울 할부지.. 아주 좋아하셨습니다 L사이즈 예쁘게 잘 맞으셔요
1,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,권여사얌,100,가을부터 초겨울까지 입기 좋은 아이템 입니다.팔이 약간 길게 나왔지만 불편한 정도는...
2,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,moontheo,100,편하게 잘입고 다닙니다. 싸게사서 부담없이 입기 좋아요
3,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,갈마동개구리,100,추천받아서 주문하고 입어보니까 예쁘네요 핏도 좋아요~~~
4,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,햄새밍턴,100,171/68에 상체발달형입니다. 일단 핏은 조금 여유있는 핏이고 이유는 전투복 위에...
...,...,...,...,...,...
40,베켄바우어 트랙탑 - 네이비 / IA4776,IA4776,뉴비_e72a31fe,100,얇아서 여름에 에어컨 바람 추울 때 입어도 좋응ㄹ것같아요
41,베켄바우어 트랙탑 - 네이비 / IA4776,IA4776,좀더깍아줘요,100,너무 마음에 듭니다 휘뚜루마뚜루 입기 딱 좋아요
42,베켄바우어 트랙탑 - 네이비 / IA4776,IA4776,뉴비_13e3057367f1,100,학교 갈때 입을려고 샀는데 데일리로 옷 걱정 안해서 좋아요
43,베켄바우어 트랙탑 - 네이비 / IA4776,IA4776,문점례,80,사이즈는 넉넉하고 색감이 너무 마음에 들었어요


In [74]:
df_musinsa_review_raw_1_3.to_csv('[이청하]review_아우터_240118.csv_1_3', index=False)

---

In [62]:
df_musinsa_review_raw

,product_names,product_ids,user_names,user_ratings,user_reviews
0,비건 스웨이드 오버핏 미니멀 인시전 스트레이트 집업 자켓 [블랙],B5,그냥주노 jus._.juno,100,"배송은 느렸지만, 상품은 정말 좋습니다.이 가격에 절대 구할 수 없는 퀄리티. 잘 ..."
1,비건 스웨이드 오버핏 미니멀 인시전 스트레이트 집업 자켓 [블랙],B5,하트도두밍,60,일반후기에 다 적어두긴했는데 저는 만족스럽지 못한 옷이었어요ㅠㅠ여자치곤 통통한 편인...
2,비건 스웨이드 오버핏 미니멀 인시전 스트레이트 집업 자켓 [블랙],B5,이으니어디우르지,100,청바지에 입기 좋아요 배송이 엄청 빨라서 놀랐어요 냄새도 별로 안나고 검정색이고 질...
3,비건 스웨이드 오버핏 미니멀 인시전 스트레이트 집업 자켓 [블랙],B5,밍코루,100,크 이쁘네요 이번 가을 아우터는 얘가 최애가 될듯요
4,비건 스웨이드 오버핏 미니멀 인시전 스트레이트 집업 자켓 [블랙],B5,노밀이,100,심플 자켓을 찾고있다가 합리적인 가격으로 구매해서 기분이 넘 좋습니다 ㅎ_ㅎ 남녀공...
...,...,...,...,...,...
40,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,기린스키위,100,정말 만족하는 제품입니다 이너로 입기 참 좋아요
41,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,Celebrity_choo,100,쌀쌀한날 편하게 입기좋고 핏도 생각하던만큼 슬림해서 좋았다 교복위에 입어도 이쁘다 ...
42,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,Mahrez1,100,적당히 집 근처 돌아댕길때 입기 좋은 가성비 아우터
43,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,앵그리성민,100,얇은 두께 치곤 따뜻합니다!! 가벼워서 사무실에서 일할때 계속 입고 있습니다 디자인...


In [63]:
df_musinsa_review_raw_1_2 = df_musinsa_review_raw
df_musinsa_review_raw_1_2

,product_names,product_ids,user_names,user_ratings,user_reviews
0,비건 스웨이드 오버핏 미니멀 인시전 스트레이트 집업 자켓 [블랙],B5,그냥주노 jus._.juno,100,"배송은 느렸지만, 상품은 정말 좋습니다.이 가격에 절대 구할 수 없는 퀄리티. 잘 ..."
1,비건 스웨이드 오버핏 미니멀 인시전 스트레이트 집업 자켓 [블랙],B5,하트도두밍,60,일반후기에 다 적어두긴했는데 저는 만족스럽지 못한 옷이었어요ㅠㅠ여자치곤 통통한 편인...
2,비건 스웨이드 오버핏 미니멀 인시전 스트레이트 집업 자켓 [블랙],B5,이으니어디우르지,100,청바지에 입기 좋아요 배송이 엄청 빨라서 놀랐어요 냄새도 별로 안나고 검정색이고 질...
3,비건 스웨이드 오버핏 미니멀 인시전 스트레이트 집업 자켓 [블랙],B5,밍코루,100,크 이쁘네요 이번 가을 아우터는 얘가 최애가 될듯요
4,비건 스웨이드 오버핏 미니멀 인시전 스트레이트 집업 자켓 [블랙],B5,노밀이,100,심플 자켓을 찾고있다가 합리적인 가격으로 구매해서 기분이 넘 좋습니다 ㅎ_ㅎ 남녀공...
...,...,...,...,...,...
40,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,기린스키위,100,정말 만족하는 제품입니다 이너로 입기 참 좋아요
41,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,Celebrity_choo,100,쌀쌀한날 편하게 입기좋고 핏도 생각하던만큼 슬림해서 좋았다 교복위에 입어도 이쁘다 ...
42,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,Mahrez1,100,적당히 집 근처 돌아댕길때 입기 좋은 가성비 아우터
43,라이트 다운 블루종 재킷 [블랙],MHJK5003-BK,앵그리성민,100,얇은 두께 치곤 따뜻합니다!! 가벼워서 사무실에서 일할때 계속 입고 있습니다 디자인...


In [64]:
df_musinsa_review_raw_1_2.to_csv('[이청하]review_아우터_240118.csv_1_2', index=False)

---

In [52]:
df_musinsa_review_raw

,product_names,product_ids,user_names,user_ratings,user_reviews
0,베이직 플리스 자켓 (U00DTJK59),2646018,은양지꽃,80,좀 커서 건조기 돌렸는데 잘 안줄어드네요 ㅋ입고다닐만해요
1,베이직 플리스 자켓 (U00DTJK59),2646018,떡볶이 사줘,80,따닷하니 좋네요 다들 한번 편하게 사서 입어보세요너무너무 좋아요
2,베이직 플리스 자켓 (U00DTJK59),2646018,oh_sori,100,일단 색깔이 보통 생각하는 그레이가 아닌 소라 색이 섞여있는 밝은 그레이 라 너무 ...
3,베이직 플리스 자켓 (U00DTJK59),2646018,773 bono,80,가볍고 편해요 헉교에서 잘 입어요 보플은 좀 생기네요
4,베이직 플리스 자켓 (U00DTJK59),2646018,".,,,,,,",100,M사이즈가 품절이라 라지로 사서 클줄알았는데 사이즈괜찮았고 예뻐용
...,...,...,...,...,...
40,유스풀 벌룬 후드집업 네이비 SJHD1327,2170224,187-1cm.,100,사이즈도 벌룬핏에 맞게 엄청 예쁘고 색감도 딱 좋아요. 사이즈가 엄청 마음에 드는 ...
41,유스풀 벌룬 후드집업 네이비 SJHD1327,2170224,sinseon_3,100,여러분들!!그냥 보이시는대로 깔끔하고 예쁩니다!!👍👍
42,유스풀 벌룬 후드집업 네이비 SJHD1327,2170224,K._jun_,100,색깔이 안예쁠줄 알았는데 생각보다 예쁘고 교복에 코디하기 좋은거 같아요. 친구들이 ...
43,유스풀 벌룬 후드집업 네이비 SJHD1327,2170224,leejen_o_군 여친,100,따듯하고 편하게입게 좋아요 걍 내 살점의 일부임


In [53]:
df_musinsa_review_raw_1 = df_musinsa_review_raw
df_musinsa_review_raw_1

,product_names,product_ids,user_names,user_ratings,user_reviews
0,베이직 플리스 자켓 (U00DTJK59),2646018,은양지꽃,80,좀 커서 건조기 돌렸는데 잘 안줄어드네요 ㅋ입고다닐만해요
1,베이직 플리스 자켓 (U00DTJK59),2646018,떡볶이 사줘,80,따닷하니 좋네요 다들 한번 편하게 사서 입어보세요너무너무 좋아요
2,베이직 플리스 자켓 (U00DTJK59),2646018,oh_sori,100,일단 색깔이 보통 생각하는 그레이가 아닌 소라 색이 섞여있는 밝은 그레이 라 너무 ...
3,베이직 플리스 자켓 (U00DTJK59),2646018,773 bono,80,가볍고 편해요 헉교에서 잘 입어요 보플은 좀 생기네요
4,베이직 플리스 자켓 (U00DTJK59),2646018,".,,,,,,",100,M사이즈가 품절이라 라지로 사서 클줄알았는데 사이즈괜찮았고 예뻐용
...,...,...,...,...,...
40,유스풀 벌룬 후드집업 네이비 SJHD1327,2170224,187-1cm.,100,사이즈도 벌룬핏에 맞게 엄청 예쁘고 색감도 딱 좋아요. 사이즈가 엄청 마음에 드는 ...
41,유스풀 벌룬 후드집업 네이비 SJHD1327,2170224,sinseon_3,100,여러분들!!그냥 보이시는대로 깔끔하고 예쁩니다!!👍👍
42,유스풀 벌룬 후드집업 네이비 SJHD1327,2170224,K._jun_,100,색깔이 안예쁠줄 알았는데 생각보다 예쁘고 교복에 코디하기 좋은거 같아요. 친구들이 ...
43,유스풀 벌룬 후드집업 네이비 SJHD1327,2170224,leejen_o_군 여친,100,따듯하고 편하게입게 좋아요 걍 내 살점의 일부임


In [54]:
df_musinsa_review_raw_1.to_csv('[이청하]review_아우터_240118.csv_1', index=False)

---

In [43]:
df_musinsa_review_raw_3 = df_musinsa_review_raw
df_musinsa_review_raw_3

,product_names,product_ids,user_names,user_ratings,user_reviews
0,072903 경량 바머 자켓,5000402643,필섭,100,저렴하게 잘 구매했네요 어디에나 잘 어울릴거같아요
1,072903 경량 바머 자켓,5000402643,김*완,100,색이 매우 이쁩니다. 환절기에 입기 좋은 따뜻함이고 가벼워서 편해요.
2,072903 경량 바머 자켓,5000402643,엠뷸란스,60,털이 너무 빠져요.. 먼지도 너무 잘 달라붙은 재질이라 입을때마다 돌돌이 써줘야 할...
3,072903 경량 바머 자켓,5000402643,김*완,100,가벼운데 따뜻하고 핏이 이쁩니다. 환절기에 입으면 좋아용
4,072903 경량 바머 자켓,5000402643,패션배울래요,100,봄이지만 아침과 저녁에 쌀쌀해서잠바 보다는 가벼유 거 입고 싶었는데저렴하게 구입해서...
...,...,...,...,...,...
30,코트 양털 후리스2X 블랙,PA001,바니바니뚜,100,두툼한 오버핏으로 떨어지는 스타일이라 좋아요
31,코트 양털 후리스2X 블랙,PA001,일어나하체하자,100,평소 XL 입는 운동 좋아하는 사람입니다~이 옷은 L로 가는게 나았을 것 같네요전 ...
32,코트 양털 후리스2X 블랙,PA001,멤버_e874915b3161,80,가성비 아주괜찮은제품이지만 오래는 못입늘거같아요
33,코트 양털 후리스2X 블랙,PA001,김동현1225,100,따뜻하고 좋아요 입기가편해서좋습니다정말좋습니다


In [44]:
df_musinsa_review_raw_3.to_csv('[이청하]review_아우터_240118.csv_3', index=False)

---

In [34]:
df_musinsa_review_raw

,product_names,product_ids,user_names,user_ratings,user_reviews
0,하버드 클래식 더플 코트 HWLC-34301 BEIGE,5004147693,백림맘,100,겨울코트는 더플코트죠! 코트지만 누빔안감이 있어 따뜻해요!! 입학시즌 졸업시즌이라 ...
1,하버드 클래식 더플 코트 HWLC-34301 BEIGE,5004147693,앙그류,100,"너무 이뻐요, 색감도 너무 이쁘고 어디에나 코디하기 좋을거 같아요."
2,하버드 클래식 더플 코트 HWLC-34301 BEIGE,5004147693,뉴비_122fcf376465,80,좋은물건감사합니다. 빠른 배송 교환 감사합니다.
0,하버드 폴라플리스 셋업 HWSU-34303 BEIGE,5004005696,두리천사,100,너무따뜻해요목둘레가 약간 답답하지만포슬포슬 보온 짱입니다
0,덤블퍼 더플 코트 (네이비) CV233OCT01_NVY,CV233OCT01_NVY,썰썰맘,100,더블코트가 밝은색상이 있어서 네이비로 찾아보다가 커버낫에 있길래 냉큼 구입했어요 ㅎ...
...,...,...,...,...,...
24,래빗 후드 집업 (화이트),2021074901831,soy_sauce_yummy,100,짱 귀여운 토끼 귀 후드 … 사이즈는 팔도 길고 꽤 큽니다. 색깔은 약간의 형광기가...
25,래빗 후드 집업 (화이트),2021074901831,Seaty,100,"마음에 드네요, 가격이 저렴합니다.배송이 빠릅니다. 디자인이 예쁘네요."
26,래빗 후드 집업 (화이트),2021074901831,son6oss,100,그저 귀여워서 샀는데너무 귀여워서 입을때마다 행복해요
27,래빗 후드 집업 (화이트),2021074901831,wofl318,100,크기랑 길이 적당해서 예쁘고 토끼귀도 너무 귀여워요


In [35]:
df_musinsa_review_raw_2 = df_musinsa_review_raw
df_musinsa_review_raw_2

,product_names,product_ids,user_names,user_ratings,user_reviews
0,하버드 클래식 더플 코트 HWLC-34301 BEIGE,5004147693,백림맘,100,겨울코트는 더플코트죠! 코트지만 누빔안감이 있어 따뜻해요!! 입학시즌 졸업시즌이라 ...
1,하버드 클래식 더플 코트 HWLC-34301 BEIGE,5004147693,앙그류,100,"너무 이뻐요, 색감도 너무 이쁘고 어디에나 코디하기 좋을거 같아요."
2,하버드 클래식 더플 코트 HWLC-34301 BEIGE,5004147693,뉴비_122fcf376465,80,좋은물건감사합니다. 빠른 배송 교환 감사합니다.
0,하버드 폴라플리스 셋업 HWSU-34303 BEIGE,5004005696,두리천사,100,너무따뜻해요목둘레가 약간 답답하지만포슬포슬 보온 짱입니다
0,덤블퍼 더플 코트 (네이비) CV233OCT01_NVY,CV233OCT01_NVY,썰썰맘,100,더블코트가 밝은색상이 있어서 네이비로 찾아보다가 커버낫에 있길래 냉큼 구입했어요 ㅎ...
...,...,...,...,...,...
24,래빗 후드 집업 (화이트),2021074901831,soy_sauce_yummy,100,짱 귀여운 토끼 귀 후드 … 사이즈는 팔도 길고 꽤 큽니다. 색깔은 약간의 형광기가...
25,래빗 후드 집업 (화이트),2021074901831,Seaty,100,"마음에 드네요, 가격이 저렴합니다.배송이 빠릅니다. 디자인이 예쁘네요."
26,래빗 후드 집업 (화이트),2021074901831,son6oss,100,그저 귀여워서 샀는데너무 귀여워서 입을때마다 행복해요
27,래빗 후드 집업 (화이트),2021074901831,wofl318,100,크기랑 길이 적당해서 예쁘고 토끼귀도 너무 귀여워요


In [36]:
df_musinsa_review_raw_2.to_csv('[이청하]review_아우터_240118.csv_2', index=False)

---

# 데이터 합치기

In [107]:
df_1 = pd.read_csv('[이청하]musinsa_아우터_240118_1.csv')
len(df_1)

100

In [108]:
df_1_2 = pd.read_csv('[이청하]musinsa_아우터_240118_1_2.csv')
len(df_1_2)

6

In [106]:
df = pd.concat([df_1, df_1_2], axis=0, ignore_index=True)
len(df)

106

In [109]:
df_1_3 = pd.read_csv('[이청하]musinsa_아우터_240118_1_3.csv')
len(df_1_3)

74

In [110]:
df = pd.concat([df, df_1_3], axis=0, ignore_index=True)
len(df)

180

In [111]:
df_2 = pd.read_csv('[이청하]musinsa_아우터_240118_2.csv')
len(df_2)

528

In [112]:
df = pd.concat([df, df_2], axis=0, ignore_index=True)
len(df)

708

In [113]:
df_2_2 = pd.read_csv('[이청하]musinsa_아우터_240118_2_2.csv')
len(df_2_2)

10

In [114]:
df = pd.concat([df, df_2_2], axis=0, ignore_index=True)
len(df)

718

In [115]:
df_2_3 = pd.read_csv('[이청하]musinsa_아우터_240118_2_3.csv')
len(df_2_3)

2

In [116]:
df = pd.concat([df, df_2_3], axis=0, ignore_index=True)
len(df)

720

In [119]:
df_3 = pd.read_csv('[이청하]musinsa_아우터_240118_3.csv')
len(df_3)

359

In [120]:
df = pd.concat([df, df_3], axis=0, ignore_index=True)
len(df)

1079

In [134]:
df.to_csv('[이청하]musinsa_아우터_240118.csv', index=False)

---

In [135]:
df_reivew_1 = pd.read_csv('[이청하]review_아우터_240118.csv_1')
len(df_reivew_1)

3325

In [136]:
df_reivew_1_2 = pd.read_csv('[이청하]review_아우터_240118.csv_1_2')
len(df_reivew_1_2)

185

In [138]:
df_review = pd.concat([df_reivew_1, df_reivew_1_2], axis=0, ignore_index=True)
len(df_review)

3510

In [139]:
df_reivew_1_3 = pd.read_csv('[이청하]review_아우터_240118.csv_1_3')
len(df_reivew_1_3)

2202

In [141]:
df_review = pd.concat([df_review, df_reivew_1_3], axis=0, ignore_index=True)
len(df_review)

7914

In [142]:
df_reivew_2 = pd.read_csv('[이청하]review_아우터_240118.csv_2')
len(df_reivew_2)

6532

In [143]:
df_review = pd.concat([df_review, df_reivew_2], axis=0, ignore_index=True)
len(df_review)

14446

In [144]:
df_reivew_2_2 = pd.read_csv('[이청하]review_아우터_240118.csv_2_2')
len(df_reivew_2_2)

209

In [145]:
df_review = pd.concat([df_review, df_reivew_2_2], axis=0, ignore_index=True)
len(df_review)

14655

In [146]:
df_reivew_2_3 = pd.read_csv('[이청하]review_아우터_240118.csv_2_3')
len(df_reivew_2_3)

25

In [147]:
df_review = pd.concat([df_review, df_reivew_2_3], axis=0, ignore_index=True)
len(df_review)

14680

In [148]:
df_reivew_3 = pd.read_csv('[이청하]review_아우터_240118.csv_3')
len(df_reivew_3)

6035

In [149]:
df_review = pd.concat([df_review, df_reivew_3], axis=0, ignore_index=True)
len(df_review)

20715

In [150]:
df_review.to_csv('[이청하]review_아우터_240118.csv', index=False)

In [153]:
df_review

,product_names,product_ids,user_names,user_ratings,user_reviews
0,베이직 플리스 자켓 (U00DTJK59),2646018,은양지꽃,80,좀 커서 건조기 돌렸는데 잘 안줄어드네요 ㅋ입고다닐만해요
1,베이직 플리스 자켓 (U00DTJK59),2646018,떡볶이 사줘,80,따닷하니 좋네요 다들 한번 편하게 사서 입어보세요너무너무 좋아요
2,베이직 플리스 자켓 (U00DTJK59),2646018,oh_sori,100,일단 색깔이 보통 생각하는 그레이가 아닌 소라 색이 섞여있는 밝은 그레이 라 너무 ...
3,베이직 플리스 자켓 (U00DTJK59),2646018,773 bono,80,가볍고 편해요 헉교에서 잘 입어요 보플은 좀 생기네요
4,베이직 플리스 자켓 (U00DTJK59),2646018,".,,,,,,",100,M사이즈가 품절이라 라지로 사서 클줄알았는데 사이즈괜찮았고 예뻐용
...,...,...,...,...,...
20710,코트 양털 후리스2X 블랙,PA001,바니바니뚜,100,두툼한 오버핏으로 떨어지는 스타일이라 좋아요
20711,코트 양털 후리스2X 블랙,PA001,일어나하체하자,100,평소 XL 입는 운동 좋아하는 사람입니다~이 옷은 L로 가는게 나았을 것 같네요전 ...
20712,코트 양털 후리스2X 블랙,PA001,멤버_e874915b3161,80,가성비 아주괜찮은제품이지만 오래는 못입늘거같아요
20713,코트 양털 후리스2X 블랙,PA001,김동현1225,100,따뜻하고 좋아요 입기가편해서좋습니다정말좋습니다


In [154]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20715 entries, 0 to 20714
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_names  20715 non-null  object
 1   product_ids    20715 non-null  object
 2   user_names     20715 non-null  object
 3   user_ratings   20715 non-null  int64 
 4   user_reviews   20715 non-null  object
dtypes: int64(1), object(4)
memory usage: 809.3+ KB


In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1079 entries, 0 to 1078
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   major_category   1079 non-null   object 
 1   middle_category  1079 non-null   object 
 2   name             1079 non-null   object 
 3   number           1079 non-null   object 
 4   limit            1079 non-null   int64  
 5   exclusive        1079 non-null   int64  
 6   target_gender    1079 non-null   int64  
 7   buy_gender       623 non-null    float64
 8   buy_age          1079 non-null   object 
 9   view             1076 non-null   float64
 10  like             1079 non-null   int64  
 11  price            1079 non-null   float64
 12  discount_rate    1079 non-null   int64  
 13  delivery_date    1077 non-null   float64
 14  rating           909 non-null    float64
 15  review           1079 non-null   int64  
 16  buy              611 non-null    float64
dtypes: float64(6),